In [1]:
import pyspark as ps
ps_sql = ps.sql.SparkSession.builder.appName("df lecture").getOrCreate()

In [1]:
import nltk

In [3]:
nltk.data.path = ['/home/nltk_data']

In [4]:
nltk.data.path

['/home/nltk_data']

In [16]:
import os
os.system("sudo chmod a+w /home/nltk_data")

os.listdir('../nltk')
os.access('/home/nltk_data', os.W_OK)

False

In [20]:
import os 
os.system("sudo chmod a+w /home")
os.system("mkdir /home/nltk_data")
nltk.download('punkt', '/home/nltk_data')

[nltk_data] Downloading package punkt to /home/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
os.listdir('../nltk_data/tokenizers/punkt')

['swedish.pickle',
 'french.pickle',
 'polish.pickle',
 'norwegian.pickle',
 'dutch.pickle',
 'portuguese.pickle',
 'danish.pickle',
 'czech.pickle',
 'greek.pickle',
 'slovene.pickle',
 'turkish.pickle',
 'english.pickle',
 'PY3',
 'finnish.pickle',
 'README',
 'spanish.pickle',
 'estonian.pickle',
 'german.pickle',
 'italian.pickle']

In [18]:
import os
os.listdir('.')

['.bash_history',
 '.ssh',
 '.bash_profile',
 '.bootstrap-end',
 'derby.log',
 'scott',
 'squash.zip',
 '.ipynb_checkpoints',
 'nltk_data',
 'Untitled.ipynb',
 '.bashrc',
 '.emacs.d',
 '.aws',
 '.ivy2',
 'anaconda',
 'jupyspark-emr.sh',
 'nlp_pipeline.pyc',
 'tokenizers',
 '.bootstrap-begin',
 'nlp_pipeline.py',
 'jupyspark-emr.sh~',
 'metastore_db',
 'nlp_pipeline.py~',
 '.local',
 '.jupyter',
 '.ipython']

In [8]:
os.environ['NLTK_DATA']

KeyError: 'NLTK_DATA'

In [1]:
ps_sql = spark

In [ ]:
#ps_sql.read.json('http://jmcauley.ucsd.edu/data/amazon/... it's in a local repo: data/reviews_Musical_Instruments_5.json.gz')
#jmcauley_url = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz'
#df_reviews = ps_sql.read.json(jmcauley_url)
# can't do http... only s3 or local
################s3a://galvanize-ds-bak/yelp_academic_dataset_business.json.gz
# https://s3.amazonaws.com/scottschwartzs3bucket/Musical_Instruments_5.json



In [2]:
jmcauley_url = 's3a://scottschwartzs3bucket/Musical_Instruments_5.json'
df_reviews = ps_sql.read.json(jmcauley_url)
df_reviews.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [3]:
print(df_reviews.count())

df_corpus = df_reviews.select('reviewText', 'overall')

df_corpus.printSchema()

from pyspark.sql.functions import count
res_test = df_corpus.groupBy("overall").agg(count("overall"))

classes_count = dict(res_test.collect())
print("class representation: {}".format(classes_count))

balanced_classsize=217
from pyspark.sql.functions import rand
dataset_neg = df_corpus.filter(df_corpus["overall"] <= 1.0).orderBy(rand()).limit(balanced_classsize)
dataset_pos = df_corpus.filter(df_corpus["overall"] >= 5.0).orderBy(rand()).limit(balanced_classsize)
df_posnegdataset = dataset_pos.union(dataset_neg)

df_posnegdataset = df_posnegdataset.withColumn("label", (df_posnegdataset['overall']-1.0)/4.0)

# notice that you need the "hadoop" user
# scp -i ~/.ssh/mysecondkey.pem nlp_pipeline.py hadoop@ec2-52-71-30-92.compute-1.amazonaws.com:~/
# and yes it worked:
# ssh -i ~/.ssh/mysecondkey.pem hadoop@ec2-52-71-30-92.compute-1.amazonaws.com

import nlp_pipeline
from nlp_pipeline import indexing_pipeline
df_output, vocab = indexing_pipeline(df_posnegdataset, inputCol="reviewText")

10261
root
 |-- reviewText: string (nullable = true)
 |-- overall: double (nullable = true)

class representation: {1.0: 217, 2.0: 250, 3.0: 772, 4.0: 2084, 5.0: 6938}
[nltk_data] Downloading package punkt to /home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Py4JJavaError: An error occurred while calling o101.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 4 times, most recent failure: Lost task 0.3 in stage 7.0 (TID 217, ip-172-31-24-136.ec2.internal): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/yarn/usercache/hadoop/appcache/application_1500142851846_0047/container_1500142851846_0047_01_000002/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/mnt/yarn/usercache/hadoop/appcache/application_1500142851846_0047/container_1500142851846_0047_01_000002/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/mnt/yarn/usercache/hadoop/appcache/application_1500142851846_0047/container_1500142851846_0047_01_000002/pyspark.zip/pyspark/worker.py", line 106, in <lambda>
    func = lambda _, it: map(mapper, it)
  File "/mnt/yarn/usercache/hadoop/appcache/appli

In [11]:
import numpy as np
import pickle as pkl
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from pyspark.mllib.clustering import KMeans
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
PUNCTUATION = set(string.punctuation)
STOPWORDS = set(stopwords.words('english'))

In [2]:
wiki_rdd = sc.textFile('s3a://galvanize-ds-bak/wiki_sample_1')

In [3]:
wiki_rdd.cache() # same as persist()
wiki_rdd.setName('wiki')

wiki MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [16]:
count = wiki_rdd.count()

In [17]:
count

374266

In [4]:
wiki_rdd_samples = sc.parallelize(wiki_rdd.take(10), 5)
wiki_no_redirect_rdd = wiki_rdd_samples.filter(lambda line: '#REDIRECT' not in line)
no_redirect = wiki_no_redirect_rdd.first()
no_redirect[:100]

u'{{Redirect2|Anarchist|Anarchists|the fictional character|Anarchist (comics)|other uses|Anarchists (d'

In [5]:
def tokenize(text):
    regex = re.compile('<.+?>|[^a-zA-Z]')
    clean_txt = regex.sub(' ', text)
    tokens = clean_txt.split()
    lowercased = [t.lower() for t in tokens]

    no_punctuation = []
    for word in lowercased:
        punct_removed = ''.join([letter for letter in word if not letter in PUNCTUATION])
        no_punctuation.append(punct_removed)
    no_stopwords = [w for w in no_punctuation if not w in STOPWORDS]
    
    STEMMER = PorterStemmer()
    stemmed = [STEMMER.stem(w) for w in no_stopwords]
    return [w for w in stemmed if w]

token_rdd = wiki_no_redirect_rdd.map(tokenize)
tokens = token_rdd.first()
tokens[:10]

In [9]:
vocab = token_rdd.flatMap(lambda x: x).distinct().collect()

def get_tf(word_lst):
    count_of_each_word = Counter(word_lst)
    doc_word_count = len(word_lst) * 1.
    return np.array([count_of_each_word[v] / doc_word_count if v in count_of_each_word else 0 for v in vocab])
    
tf_rdd = token_rdd.map(get_tf)
tf_first = tf_rdd.first()
tf_first[:20]

array([  2.60416667e-04,   5.20833333e-04,   3.90625000e-04,
         5.20833333e-04,   1.95312500e-04,   6.51041667e-05,
         6.51041667e-05,   1.30208333e-04,   5.20833333e-04,
         6.51041667e-05,   6.51041667e-05,   1.30208333e-04,
         6.51041667e-05,   6.51041667e-05,   1.30208333e-04,
         3.25520833e-04,   6.51041667e-05,   6.51041667e-05,
         1.30208333e-04,   4.55729167e-04])

In [12]:
total_doc_count = tf_rdd.count()
times_words_in_doc = tf_rdd.map(lambda tf_lst: ((np.array(tf_lst) > 0) + 0)).sum()
idf = np.log(total_doc_count / times_words_in_doc)

tfidf_rdd = tf_rdd.map(lambda tf_vec: tf_vec * idf)
tfidf_rdd.cache()
tfidf_rdd.setName('tfidf')

In [17]:
model = KMeans.train(tfidf_rdd, 2)
centriods = model.centers

In [18]:
centriods

[array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]